In [1]:
import cv2
import sys
import csv
import time
import numpy as np

In [2]:
### Load training images and labels

imageDirectory = './2022Fimgs/'

with open(imageDirectory + 'train.txt', 'r') as f:
    reader = csv.reader(f)
    lines = list(reader)

In [3]:
# this line reads in all images listed in the file in GRAYSCALE, and resizes them to 33x25 pixels
train = np.array([np.array(cv2.resize(cv2.imread(imageDirectory +lines[i][0]+".png",0),(33,25))) for i in range(len(lines))])

In [4]:
# here we reshape each image into a long vector and ensure the data type is a float (which is what KNN wants)
train_data = train.flatten().reshape(len(lines), 33*25)
train_data = train_data.astype(np.float32)

In [5]:
# read in training labels
train_labels = np.array([np.int32(lines[i][1]) for i in range(len(lines))])

In [6]:
### Train classifier
knn = cv2.ml.KNearest_create()
knn.train(train_data, cv2.ml.ROW_SAMPLE, train_labels)

True

In [7]:
if(__debug__):
	Title_images = 'Original Image'
	Title_resized = 'Image Resized'
	cv2.namedWindow( Title_images, cv2.WINDOW_AUTOSIZE )

qt.qpa.plugin: Could not find the Qt platform plugin "wayland" in "/home/yash/miniconda3/lib/python3.10/site-packages/cv2/qt/plugins"


In [8]:
imageDirectory = './2022Fimgs/'

### Run test images
with open(imageDirectory + 'test.txt', 'r') as f:
    reader = csv.reader(f)
    lines = list(reader)

In [ ]:
correct = 0.0
confusion_matrix = np.zeros((6,6))

k = 21

for i in range(len(lines)):
    original_img = cv2.imread(imageDirectory+lines[i][0]+".png",0)
    test_img = np.array(cv2.resize(cv2.imread(imageDirectory+lines[i][0]+".png",0),(33,25)))
    if(__debug__):
        cv2.imshow(Title_images, original_img)
        cv2.imshow(Title_resized, test_img)
        key = cv2.waitKey()
        if key==27:    # Esc key to stop
            break
    test_img = test_img.flatten().reshape(1, 33*25)
    test_img = test_img.astype(np.float32)

    test_label = np.int32(lines[i][1])

    ret, results, neighbours, dist = knn.findNearest(test_img, k)

    if test_label == ret:
        print(str(lines[i][0]) + " Correct, " + str(ret))
        correct += 1
        confusion_matrix[np.int32(ret)][np.int32(ret)] += 1
    else:
        confusion_matrix[test_label][np.int32(ret)] += 1
        
        print(str(lines[i][0]) + " Wrong, " + str(test_label) + " classified as " + str(ret))
        print("\tneighbours: " + str(neighbours))
        print("\tdistances: " + str(dist))

In [ ]:
print("\n\nTotal accuracy: " + str(correct/len(lines)))
print(confusion_matrix)